In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

import os
import cv2
import torch
import scipy
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torchvision.transforms.functional as TF
from PIL import Image
import matplotlib.pyplot as plt
from scipy import io as sio
from scipy import ndimage, misc
import csv
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [0]:
data_dir = "ResNet18_data/train"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "alexnet"

# Number of classes in the dataset
num_classes = 8

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

In [0]:
def read_images(images_path, number_of_images):
  images = []
  for i in range(number_of_images):
    print(i+1)
    path = images_path + '/' + str(i+1) + '.jpg'
    image = scipy.misc.imread(path)
    images.append(image)
  images = np.array(images)
  print(images.shape)
  return images

In [0]:
def read_labels(labels_path):
	with open(labels_path,'rU') as csvfile:
		csvfile = csv.reader(csvfile, delimiter=',')
		csvdata = list(csvfile)
		labels = map(int, csvdata[0])
		return list(labels)

In [0]:
# train_images_path = '/content/drive/My Drive/cv_assign3_dataset/train'
# train_images = read_images(train_images_path, 1888)
# train_images.shape
# train_labels_path = '/content/drive/My Drive/cv_assign3_dataset/train_labels.csv'
# train_labels = read_labels(train_labels_path)
# print(len(train_labels))
# # print train_labels

# test_images_path = '/content/drive/My Drive/cv_assign3_dataset/test'
# test_images = read_images(test_images_path, 800)
# test_images.shape
# test_labels_path = '/content/drive/My Drive/cv_assign3_dataset/test_labels.csv'
# test_labels = read_labels(test_labels_path)
# print(len(test_labels))

In [0]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [0]:
alexnet = models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.torch/models/alexnet-owt-4df8aa71.pth
244418560it [00:08, 29397727.17it/s]


In [0]:
alexnet.classifier[6] = nn.Linear(4096,8)

In [0]:
alexnet.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p

In [0]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

  

In [0]:
train_labels = np.array(pd.read_csv('/content/drive/My Drive/ResNet18_data/train_labels.csv',header=None))[0]
test_labels = np.array(pd.read_csv('/content/drive/My Drive/ResNet18_data/test_labels.csv',header=None))[0]

In [0]:
class FaceLandmarksDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self,root_dir,total_count, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.total_count = total_count
        self.csv_file = np.array(pd.read_csv(csv_file,header=None))[0]
        self.transform = transform

    def __len__(self):
        return self.total_count

    def __getitem__(self, image_no):
        img_name = os.path.join(self.root_dir,
                                str(image_no+1)+".jpg")
        image = Image.open(img_name)
        sample = {'image': image, 'label': self.csv_file[image_no]-1}

        if self.transform:
            sample['image'] = self.transform(sample['image'])

        return sample

In [0]:
trainDataset = FaceLandmarksDataset('/content/drive/My Drive/ResNet18_data/train',1888,'/content/drive/My Drive/ResNet18_data/train_labels.csv',transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]))

testDataset = FaceLandmarksDataset('/content/drive/My Drive/ResNet18_data/test',800,'/content/drive/My Drive/ResNet18_data/test_labels.csv',transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]))

In [0]:
# Initialize the model for this run
model_ft, input_size = initialize_model("alexnet", 8, feature_extract, use_pretrained=True)

In [0]:
# Print the model we just instantiated
print(model_ft)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p

In [0]:
print("Initializing Datasets and Dataloaders...")

trainDataloader = torch.utils.data.DataLoader(trainDataset,batch_size = 8, shuffle = True, num_workers=4)
testDataloader = torch.utils.data.DataLoader(testDataset,batch_size = 8, shuffle = True, num_workers=4)
Dataloaders_dict = {trainDataloader,testDataloader}
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


In [0]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)


Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [0]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
#model_ft, hist = train_model(model_ft,trainDataloader,testDataloader, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="alexnet"))

In [0]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainDataloader, 0):
        # get the inputs
        inputs, labels = data['image'],data['label']
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer_ft.zero_grad()
        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_ft.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')

[1,    10] loss: 0.247
[1,    20] loss: 0.233
[1,    30] loss: 0.434
[1,    40] loss: 0.205
[1,    50] loss: 0.311
[1,    60] loss: 0.126
[1,    70] loss: 0.634
[1,    80] loss: 0.210
[1,    90] loss: 0.247
[1,   100] loss: 0.519
[1,   110] loss: 0.418
[1,   120] loss: 0.156
[1,   130] loss: 0.474
[1,   140] loss: 0.244
[1,   150] loss: 0.479
[1,   160] loss: 0.178
[1,   170] loss: 0.221
[1,   180] loss: 0.376
[1,   190] loss: 0.354
[1,   200] loss: 0.294
[1,   210] loss: 0.163
[1,   220] loss: 0.107
[1,   230] loss: 0.381
[2,    10] loss: 0.526
[2,    20] loss: 0.400
[2,    30] loss: 0.176
[2,    40] loss: 0.216
[2,    50] loss: 0.333
[2,    60] loss: 0.182
[2,    70] loss: 0.530
[2,    80] loss: 0.281
[2,    90] loss: 0.404
[2,   100] loss: 0.237
[2,   110] loss: 0.280
[2,   120] loss: 0.286
[2,   130] loss: 0.268
[2,   140] loss: 0.464
[2,   150] loss: 0.236
[2,   160] loss: 0.293
[2,   170] loss: 0.358
[2,   180] loss: 0.270
[2,   190] loss: 0.302
[2,   200] loss: 0.252
[2,   210] 

In [0]:
running_loss

2.720498561859131

In [0]:
correct = 0
total = 0
with torch.no_grad():
    for i, data in enumerate(testDataloader, 0):
        # get the inputs
        inputs, labels = data['image'],data['label']
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model_ft(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('Accuracy of the network on the 800 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 800 test images: 90 %
